In [1]:
import pandas as pd
from thunderpack import ThunderReader
from tqdm import tqdm
import re

In [2]:
reader = ThunderReader('/home/jsearle/bigDrive/Dropbox/zz_EHR_Thunderpacks/BIDMC/thunderpack_icd_9_10_nax_1m_BIDMC')
key_length = len(list(reader.keys()))
print(key_length)

36


In [4]:
# filter by below icd codes
code_regex = '^(I60|430)'
dfs = []
for i in tqdm(range(1, key_length + 1)):
    df = reader[f'ICD_partition_{i}']
    df = df[df['DiagnosisCodeWithDots'].astype(str).str.match(code_regex, flags = re.I)]
    dfs.append(df)

100%|██████████| 36/36 [00:39<00:00,  1.10s/it]


In [5]:
# show num of total SAH icd codes given
# show example of df
filtered_icd_df = pd.concat(dfs, axis=0, ignore_index=True)
print(len(filtered_icd_df))
filtered_icd_df.head()

4203


,BDSPPatientID,BDSPEncounterID,DiagnosisSequenceNumber,DiagnosisCode,DiagnosisPoaInd,DiagnosisCodeWithDots,ShortDescription,LongDescription,DiagnosisType,AdmissionDate,DischargeDate,BDSPLastModifiedDTS,code_type
0,150000004,132917345,1,430,Y,430,SUBARACHNOID HEMORRHAGE,SUBARACHNOID HEMORRHAGE ...,ICD9,2012-11-05,2012-12-14,2023-07-17 10:28:55.1290270,ICD9
1,150000137,133023864,1,430,Y,430,SUBARACHNOID HEMORRHAGE,SUBARACHNOID HEMORRHAGE ...,ICD9,2013-06-14,2013-07-24,2023-07-17 10:28:55.1290270,ICD9
2,150000004,557677029,1,430,Y,430,SUBARACHNOID HEMORRHAGE,SUBARACHNOID HEMORRHAGE ...,ICD9,2012-11-05,2012-11-05,2023-07-17 10:28:55.1290270,ICD9
3,150063730,558573586,1,430,Y,430,SUBARACHNOID HEMORRHAGE,SUBARACHNOID HEMORRHAGE ...,ICD9,2014-03-07,2014-03-07,2023-07-17 11:58:29.1092480,ICD9
4,150000004,922613611,3,430,None,430,SUBARACHNOID HEMORRHAGE,SUBARACHNOID HEMORRHAGE ...,ICD9,2013-02-22,2013-02-22,2023-07-17 10:28:55.1290270,ICD9


In [8]:
# clean up df, keep only relevant info
keepColumns = ['BDSPPatientID', 'AdmissionDate', 'DiagnosisCodeWithDots']
clean_icd_df = filtered_icd_df[keepColumns]
clean_icd_df.head()

,BDSPPatientID,AdmissionDate,DiagnosisCodeWithDots
0,150000004,2012-11-05,430
1,150000137,2013-06-14,430
2,150000004,2012-11-05,430
3,150063730,2014-03-07,430
4,150000004,2013-02-22,430


In [11]:
# rename columns
rename_dict = { 
    'DiagnosisCodeWithDots': 'ICD', 
}

clean_icd_df = clean_icd_df.rename(columns=rename_dict)

clean_icd_df.head()


,BDSPPatientID,AdmissionDate,ICD
0,150000004,2012-11-05,430
1,150000137,2013-06-14,430
2,150000004,2012-11-05,430
3,150063730,2014-03-07,430
4,150000004,2013-02-22,430


In [12]:
# save df as csv
clean_icd_df.to_csv('1_icd_pos_df.csv', index=False)